<a href="https://colab.research.google.com/github/srkwon/analysis/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B1%B0%EB%9E%98%EC%86%8C_%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%BD%94%EB%93%9C_%EA%B5%AC%EC%84%B1_20250629_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BeautifulSoup 기반 크롤링


In [10]:
import requests
from bs4 import BeautifulSoup
import time

base_url = "https://kdx.kr"

start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

print(f"리스트 페이지 접근: {start_url}")
res = requests.get(start_url)
if res.status_code != 200:
    print("❌ 리스트 페이지 로드 실패")
    exit()

soup = BeautifulSoup(res.text, "html.parser")

# ✅ BeautifulSoup는 비표준 구조를 교정 → li 안에 a가 들어가게 됨
list_items = soup.select("ul.card-list li.list-item")

print(f"✅ 리스트 아이템 개수: {len(list_items)}")

links = []

for li in list_items:
    a_tag = li.find("a", href=True)
    if a_tag:
        href = a_tag["href"]
        links.append(href)
        print(base_url + href)

print(f"\n✅ 최종 추출된 링크 개수: {len(links)}")


data_list = []

for idx, li in enumerate(items, start=1):
    # li의 부모 <a> 태그 탐색
    parent_a = li.find_parent("a")
    if not parent_a or not parent_a.get("href"):
        print(f"⚠️ [{idx}] 링크 없음, 건너뜀")
        continue

    href = parent_a.get("href")
    detail_url = base_url + href
    print(f"\n--- [{idx}] 상세 페이지 접근: {detail_url}")

    detail_res = requests.get(detail_url)
    if detail_res.status_code != 200:
        print(f"❌ [{idx}] 상세 페이지 로드 실패: {detail_url}")
        continue

    detail_soup = BeautifulSoup(detail_res.text, "html.parser")

    # 총 구매금액
    purchase_elem = detail_soup.select_one("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
    if purchase_elem:
        purchase_amount = purchase_elem.get_text(strip=True)
        print(f"💰 총 구매금액: {purchase_amount}")
    else:
        purchase_amount = "N/A"
        print("⚠️ 총 구매금액 정보 없음")

    # 스키마(샘플 데이터 테이블)
    schema_table = detail_soup.select_one("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap.expand > div.sample_data > table")
    if schema_table:
        schema_text = schema_table.get_text(separator=" | ", strip=True)
        print(f"📄 스키마(샘플 데이터) 요약: {schema_text[:100]}...")
    else:
        schema_text = "N/A"
        print("⚠️ 스키마 정보 없음")

    # 결과 저장
    data_list.append({
        "url": detail_url,
        "purchase_amount": purchase_amount,
        "schema": schema_text,
    })

    time.sleep(1)

print("\n✅ 전체 크롤링 완료")
print(f"총 수집된 데이터 수: {len(data_list)}")
for d in data_list[:3]:
    print("\n📝 예시 데이터")
    print(d)


리스트 페이지 접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
✅ 리스트 아이템 개수: 0

✅ 최종 추출된 링크 개수: 0

✅ 전체 크롤링 완료
총 수집된 데이터 수: 0


# Selenium 기반 크롤링

In [8]:
!apt-get update
!apt install -y wget unzip curl

# 크롬 설치
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -fy install

# 크롬 버전 확인
!google-chrome --version

# 크롬 버전에 맞는 드라이버 버전 가져오기
!CHROME_VERSION=$(google-chrome --version | cut -d ' ' -f 3 | cut -d '.' -f 1) \
 && echo "Using Chrome major version: $CHROME_VERSION" \
 && DRIVER_VERSION=$(curl -sS "https://chromedriver.storage.googleapis.com/LATEST_RELEASE_${CHROME_VERSION}") \
 && echo "Matching chromedriver version: $DRIVER_VERSION" \
 && wget -O /tmp/chromedriver.zip "https://chromedriver.storage.googleapis.com/${DRIVER_VERSION}/chromedriver_linux64.zip" \
 && unzip /tmp/chromedriver.zip chromedriver -d /usr/bin/

!chmod +x /usr/bin/chromedriver
!pip install selenium


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,212 B]
Hit:12 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 3,037 B in 2s (1,366 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repos

In [9]:
# 🌟 셀 2: 크롤링 코드

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# URL 설정
base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

# 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.binary_location = "/usr/bin/google-chrome"

# 드라이버 경로 설정
service = Service("/usr/bin/chromedriver")

# 드라이버 초기화
driver = webdriver.Chrome(service=service, options=chrome_options)

print(f"접근: {start_url}")
driver.get(start_url)
time.sleep(2)

# 상품 링크 추출
a_tags = driver.find_elements(By.CSS_SELECTOR, "ul.card-list > a")
print(f"세부 링크 개수: {len(a_tags)}")

links = []

for idx, a in enumerate(a_tags, start=1):
    href = a.get_attribute("href")
    if href:
        full_url = base_url + href
        print(f"[{idx}] {full_url}")
        links.append(full_url)

# 첫 번째 상세 페이지 예제 접근
if links:
    driver.get(links[0])
    time.sleep(1.5)

    try:
        purchase_elem = driver.find_element(By.CSS_SELECTOR, "#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
        purchase_amount = purchase_elem.text.strip()
        print(f"\n💰 총 구매금액: {purchase_amount}")
    except:
        print("⚠️ 총 구매금액 정보 없음")

    try:
        schema_elem = driver.find_element(By.CSS_SELECTOR, "#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap.expand > div.sample_data > table")
        schema_text = schema_elem.text.strip()
        print(f"📄 스키마 요약: {schema_text[:100]}...")
    except:
        print("⚠️ 스키마 정보 없음")

driver.quit()
print("\n✅ Selenium 크롤링 완료")


WebDriverException: Message: Service /usr/bin/chromedriver unexpectedly exited. Status code was: 1


# Playwright 기반 크롤링

In [10]:
!pip install playwright
!playwright install


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 MB 21.1 MB/s eta 0:00:00
171.6 MiB [] 0% 0.0s171.6 MiB [] 0% 94.9s171.6 MiB [] 0% 108.1s171.6 MiB [] 0% 71.3s171.6 MiB [] 0% 52.9s171.6 MiB [] 0% 40.6s171.6 MiB [] 0% 31.2s171.6 MiB [] 0% 22.2s171.6 MiB [] 0% 16.0s171.6 MiB [] 1% 11.4s171.6 MiB [] 2% 8.4s171.6 MiB [] 2% 7.0s171.6 MiB [] 3% 6.3s171.6 MiB [] 3% 5.9s171.6 MiB [] 4% 5.5s171.6 MiB [] 4% 5.6s171.6 MiB [] 5% 5.1s171.6 MiB [] 5% 5.5s171.6 MiB [] 6% 4.9s171.6 MiB [] 6% 4.7s171.6 MiB [] 7% 4.4s171.6 MiB [] 7% 4.3s171.6 MiB [] 8% 4.2s171.6 MiB [] 9% 4.0s171.6 MiB [] 10% 3.7s171.6 MiB [] 11% 3.6s171.6 MiB [] 12% 3.4s171.6 MiB [] 13% 3.2s171.6 MiB [] 14% 3.0s171.6 MiB [] 15% 2.9s171.6 MiB [] 16% 2.9s171.6 MiB [] 16% 2.8s171.6 MiB [] 17% 2.7s171.6 MiB [] 18% 2.6s171.6 MiB [] 19% 2.5s171.6 MiB [] 21% 2.4s171.6 MiB [] 21% 2.3s171.6 MiB [] 21% 2.4s171.6 MiB [] 22% 2.4s171.6 MiB [] 23% 2.3s171.6 MiB [] 24% 2.2s171.6 MiB [] 25% 2.1s171.6 MiB [] 27% 2.0s171.6 MiB [] 28% 2.0s171.6 MiB [

In [12]:
import asyncio
from playwright.async_api import async_playwright

base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        print(f"접근: {start_url}")
        await page.goto(start_url)
        await page.wait_for_timeout(3000)  # 3초 대기

        a_tags = await page.query_selector_all("ul.card-list > a")
        print(f"세부 링크 개수: {len(a_tags)}")

        links = []
        for idx, a in enumerate(a_tags, start=1):
            href = await a.get_attribute("href")
            if href:
                full_url = base_url + href
                print(f"[{idx}] {full_url}")
                links.append(full_url)

        # 첫 번째 상세 페이지 예제
        if links:
            await page.goto(links[0])
            await page.wait_for_timeout(3000)

            try:
                purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                purchase_amount = await purchase_elem.inner_text()
                print(f"\n💰 총 구매금액: {purchase_amount.strip()}")
            except:
                print("⚠️ 총 구매금액 정보 없음")

            try:
                schema_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap.expand > div.sample_data > table")
                schema_text = await schema_elem.inner_text()
                print(f"📄 스키마 요약: {schema_text.strip()[:100]}...")
            except:
                print("⚠️ 스키마 정보 없음")

        await browser.close()
        print("\n✅ Playwright 크롤링 완료")

# 코랩 환경에서 asyncio 루프 돌리기
await run()


접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
세부 링크 개수: 4
[1] https://kdx.kr/data/view/41601
[2] https://kdx.kr/data/view/41600
[3] https://kdx.kr/data/view/41599
[4] https://kdx.kr/data/view/41475

💰 총 구매금액: 22,000,000원
⚠️ 스키마 정보 없음

✅ Playwright 크롤링 완료


In [15]:
import asyncio
from playwright.async_api import async_playwright

base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        print(f"접근: {start_url}")
        await page.goto(start_url)
        await page.wait_for_timeout(3000)

        a_tags = await page.query_selector_all("ul.card-list > a")
        print(f"세부 링크 개수: {len(a_tags)}")

        links = []
        for idx, a in enumerate(a_tags, start=1):
            href = await a.get_attribute("href")
            if href:
                full_url = base_url + href
                print(f"[{idx}] {full_url}")
                links.append(full_url)

        # ✅ 각 세부 페이지 반복
        for detail_idx, detail_url in enumerate(links, start=1):
            print(f"\n--- [{detail_idx}] 상세 페이지 접근: {detail_url}")
            await page.goto(detail_url)
            await page.wait_for_timeout(2000)

            # 총 구매금액
            try:
                purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                purchase_amount = await purchase_elem.inner_text()
                print(f"💰 총 구매금액: {purchase_amount.strip()}")
            except:
                purchase_amount = "N/A"
                print("⚠️ 총 구매금액 정보 없음")

            # 데이터 스키마 컬럼 추출
            try:
                th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                schema_columns = []
                for th in th_elements:
                    text = await th.inner_text()
                    schema_columns.append(text.strip())

                schema_text_clean = " | ".join(schema_columns)
                print(f"📄 데이터 스키마(컬럼): {schema_text_clean}")
            except:
                schema_text_clean = "N/A"
                print("⚠️ 데이터 스키마 정보 없음")

        await browser.close()
        print("\n✅ Playwright 크롤링 완료")

await run()


접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
세부 링크 개수: 4
[1] https://kdx.kr/data/view/41601
[2] https://kdx.kr/data/view/41600
[3] https://kdx.kr/data/view/41599
[4] https://kdx.kr/data/view/41475

--- [1] 상세 페이지 접근: https://kdx.kr/data/view/41601
💰 총 구매금액: 22,000,000원
📄 데이터 스키마(컬럼): STD_YM | BLCK_SP_CD | CTPV_CD | CTPV_NM | CTGG_CD | CTGG_NM | ADSTRD_CD | ADSTRD_NM | SEXD_CD | AGGRD_CD | JOBG_CD | TOT_ASST_AMT | NET_ASST_AMT

--- [2] 상세 페이지 접근: https://kdx.kr/data/view/41600
💰 총 구매금액: 17,600,000원
📄 데이터 스키마(컬럼): STD_YM | BLCK_SP_CD | CTPV_CD | CTPV_NM | CTGG_CD | CTGG_NM | SEXD_CD | AGGRD_CD | JOBG_CD | TOT_ASST_AMT | NET_ASST_AMT

--- [3] 상세 페이지 접근: https://kdx.kr/data/view/41599
⚠️ 총 구매금액 정보 없음
📄 데이터 스키마(컬럼): STD_YM | BLCK_SP_CD | CTPV_CD | CTPV_NM | TOT_ASST_AMT | NET_ASST_AMT | ONW_HOUS_RATIO | PLU_HOUS_RATIO | APT_RES_RATIO

--- [4] 상세 페이지 접근: https://kdx.kr/data/view/41475
💰 총 구매금액: 22,000,000원
📄 데이터 스키마(컬럼): STD_YM | BLCK_SP_CD

In [16]:
import asyncio
import json
import re
from playwright.async_api import async_playwright

base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        print(f"접근: {start_url}")
        await page.goto(start_url)
        await page.wait_for_timeout(3000)

        a_tags = await page.query_selector_all("ul.card-list > a")
        print(f"세부 링크 개수: {len(a_tags)}")

        links = []
        for idx, a in enumerate(a_tags, start=1):
            href = await a.get_attribute("href")
            if href:
                full_url = base_url + href
                print(f"[{idx}] {full_url}")
                links.append(full_url)

        # 파일 준비
        output_file = "output.jsonl"
        with open(output_file, "w", encoding="utf-8") as f:
            # 각 상세 페이지 접근
            for detail_idx, detail_url in enumerate(links, start=1):
                print(f"\n--- [{detail_idx}] 상세 페이지 접근: {detail_url}")
                await page.goto(detail_url)
                await page.wait_for_timeout(2000)

                # 총 구매금액
                try:
                    purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                    purchase_text = await purchase_elem.inner_text()
                    # 문자열에서 숫자 추출
                    price_num = int(re.sub(r"[^\d]", "", purchase_text))
                    print(f"💰 총 구매금액(숫자): {price_num}")
                except:
                    price_num = 0
                    print("⚠️ 총 구매금액 정보 없음, 0 처리")

                # 데이터 스키마 컬럼 추출
                try:
                    th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                    schema_columns = []
                    for th in th_elements:
                        text = await th.inner_text()
                        schema_columns.append(text.strip())
                    print(f"📄 컬럼: {schema_columns}")
                except:
                    schema_columns = []
                    print("⚠️ 데이터 스키마 정보 없음, 빈 리스트 처리")

                # JSON 객체 생성
                data = {
                    "columns": schema_columns,
                    "price": price_num
                }

                # 파일에 한 줄씩 기록
                f.write(json.dumps(data, ensure_ascii=False) + "\n")

        await browser.close()
        print(f"\n✅ 크롤링 및 파일 작성 완료! ➜ {output_file}")

await run()


접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
세부 링크 개수: 4
[1] https://kdx.kr/data/view/41601
[2] https://kdx.kr/data/view/41600
[3] https://kdx.kr/data/view/41599
[4] https://kdx.kr/data/view/41475

--- [1] 상세 페이지 접근: https://kdx.kr/data/view/41601
💰 총 구매금액(숫자): 22000000
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'CTGG_CD', 'CTGG_NM', 'ADSTRD_CD', 'ADSTRD_NM', 'SEXD_CD', 'AGGRD_CD', 'JOBG_CD', 'TOT_ASST_AMT', 'NET_ASST_AMT']

--- [2] 상세 페이지 접근: https://kdx.kr/data/view/41600
💰 총 구매금액(숫자): 17600000
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'CTGG_CD', 'CTGG_NM', 'SEXD_CD', 'AGGRD_CD', 'JOBG_CD', 'TOT_ASST_AMT', 'NET_ASST_AMT']

--- [3] 상세 페이지 접근: https://kdx.kr/data/view/41599
⚠️ 총 구매금액 정보 없음, 0 처리
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'TOT_ASST_AMT', 'NET_ASST_AMT', 'ONW_HOUS_RATIO', 'PLU_HOUS_RATIO', 'APT_RES_RATIO']

--- [4] 상세 페이지 접근: https://kdx.kr/data/view/41475
💰 총 구매금액(숫자): 22000000
📄 컬럼: ['ST

In [17]:
import asyncio
import json
import re
from playwright.async_api import async_playwright

base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        print(f"접근: {start_url}")
        await page.goto(start_url)
        await page.wait_for_timeout(3000)

        a_tags = await page.query_selector_all("ul.card-list > a")
        print(f"세부 링크 개수: {len(a_tags)}")

        links = []
        for idx, a in enumerate(a_tags, start=1):
            href = await a.get_attribute("href")
            if href:
                full_url = base_url + href
                print(f"[{idx}] {full_url}")
                links.append(full_url)

        output_file = "output.jsonl"
        with open(output_file, "w", encoding="utf-8") as f:
            for detail_idx, detail_url in enumerate(links, start=1):
                print(f"\n--- [{detail_idx}] 상세 페이지 접근: {detail_url}")
                await page.goto(detail_url)
                await page.wait_for_timeout(2000)

                # 총 구매금액
                try:
                    purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                    purchase_text = await purchase_elem.inner_text()
                    price_num = int(re.sub(r"[^\d]", "", purchase_text))
                    print(f"💰 총 구매금액(숫자): {price_num}")
                except:
                    price_num = 0
                    print("⚠️ 총 구매금액 정보 없음, 0 처리")

                # 데이터 스키마 컬럼 추출
                try:
                    th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                    schema_columns = []
                    for th in th_elements:
                        text = await th.inner_text()
                        schema_columns.append(text.strip())
                    print(f"📄 컬럼: {schema_columns}")
                except:
                    schema_columns = []
                    print("⚠️ 데이터 스키마 정보 없음, 빈 리스트 처리")

                # 상품명
                try:
                    name_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(2) > td")
                    name_text = await name_elem.inner_text()
                except:
                    name_text = "N/A"

                # 카테고리
                try:
                    category_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(1) > td")
                    category_text = await category_elem.inner_text()
                except:
                    category_text = "N/A"

                # 데이터 포맷
                try:
                    format_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(6) > td")
                    format_text = await format_elem.inner_text()
                except:
                    format_text = "N/A"

                # 데이터 사이즈
                try:
                    size_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(7) > td")
                    size_text = await size_elem.inner_text()
                except:
                    size_text = "N/A"

                # JSON 객체 구성
                data = {
                    "columns": schema_columns,
                    "price": price_num,
                    "name": name_text.strip(),
                    "category": category_text.strip(),
                    "format": format_text.strip(),
                    "size": size_text.strip()
                }

                # 파일에 기록
                f.write(json.dumps(data, ensure_ascii=False) + "\n")

        await browser.close()
        print(f"\n✅ 크롤링 및 파일 작성 완료! ➜ {output_file}")

await run()


접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
세부 링크 개수: 4
[1] https://kdx.kr/data/view/41601
[2] https://kdx.kr/data/view/41600
[3] https://kdx.kr/data/view/41599
[4] https://kdx.kr/data/view/41475

--- [1] 상세 페이지 접근: https://kdx.kr/data/view/41601
💰 총 구매금액(숫자): 22000000
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'CTGG_CD', 'CTGG_NM', 'ADSTRD_CD', 'ADSTRD_NM', 'SEXD_CD', 'AGGRD_CD', 'JOBG_CD', 'TOT_ASST_AMT', 'NET_ASST_AMT']

--- [2] 상세 페이지 접근: https://kdx.kr/data/view/41600
💰 총 구매금액(숫자): 17600000
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'CTGG_CD', 'CTGG_NM', 'SEXD_CD', 'AGGRD_CD', 'JOBG_CD', 'TOT_ASST_AMT', 'NET_ASST_AMT']

--- [3] 상세 페이지 접근: https://kdx.kr/data/view/41599
⚠️ 총 구매금액 정보 없음, 0 처리
📄 컬럼: ['STD_YM', 'BLCK_SP_CD', 'CTPV_CD', 'CTPV_NM', 'TOT_ASST_AMT', 'NET_ASST_AMT', 'ONW_HOUS_RATIO', 'PLU_HOUS_RATIO', 'APT_RES_RATIO']

--- [4] 상세 페이지 접근: https://kdx.kr/data/view/41475
💰 총 구매금액(숫자): 22000000
📄 컬럼: ['ST

In [18]:
import asyncio
import json
import re
from playwright.async_api import async_playwright

base_url = "https://kdx.kr"
start_url = "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(3000)

        output_file = "output.jsonl"
        with open(output_file, "w", encoding="utf-8") as f:

            # 페이지네이션 버튼 전체 가져오기
            page_buttons = await page.query_selector_all("#pagination > a.tui-page-btn")
            total_pages = len(page_buttons)
            print(f"총 페이지 개수: {total_pages}")

            # 페이지 순회
            for page_idx in range(1, total_pages + 1):
                print(f"\n✅ [페이지 {page_idx}] 수집 시작")
                await page.wait_for_timeout(2000)

                a_tags = await page.query_selector_all("ul.card-list > a")
                print(f"상품 링크 개수: {len(a_tags)}")

                links = []
                for a in a_tags:
                    href = await a.get_attribute("href")
                    if href:
                        full_url = base_url + href
                        links.append(full_url)

                for detail_idx, detail_url in enumerate(links, start=1):
                    print(f"\n--- [페이지 {page_idx}, 상품 {detail_idx}] 상세 페이지: {detail_url}")
                    await page.goto(detail_url)
                    await page.wait_for_timeout(1500)

                    try:
                        purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                        purchase_text = await purchase_elem.inner_text()
                        price_num = int(re.sub(r"[^\d]", "", purchase_text))
                    except:
                        price_num = 0

                    try:
                        th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                        schema_columns = [await th.inner_text() for th in th_elements]
                        schema_columns = [col.strip() for col in schema_columns]
                    except:
                        schema_columns = []

                    try:
                        name_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(2) > td")
                        name_text = await name_elem.inner_text()
                    except:
                        name_text = "N/A"

                    try:
                        category_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(1) > td")
                        category_text = await category_elem.inner_text()
                    except:
                        category_text = "N/A"

                    try:
                        format_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(6) > td")
                        format_text = await format_elem.inner_text()
                    except:
                        format_text = "N/A"

                    try:
                        size_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(7) > td")
                        size_text = await size_elem.inner_text()
                    except:
                        size_text = "N/A"

                    data = {
                        "columns": schema_columns,
                        "price": price_num,
                        "name": name_text.strip(),
                        "category": category_text.strip(),
                        "format": format_text.strip(),
                        "size": size_text.strip()
                    }

                    f.write(json.dumps(data, ensure_ascii=False) + "\n")

                    await page.goto(start_url)
                    await page.wait_for_timeout(1500)

                # 다음 페이지 클릭
                if page_idx < total_pages:
                    next_btn = await page.query_selector(f"#pagination > a.tui-page-btn:nth-child({page_idx + 3})")  # index 조정 필요
                    if next_btn:
                        await next_btn.click()
                        await page.wait_for_timeout(3000)
                    else:
                        print("\n⚡ 다음 버튼 없음 — 종료")
                        break

        await browser.close()
        print(f"\n✅ 전체 페이지 크롤링 및 파일 저장 완료 ➜ {output_file}")

await run()


총 페이지 개수: 7

✅ [페이지 1] 수집 시작
상품 링크 개수: 4

--- [페이지 1, 상품 1] 상세 페이지: https://kdx.kr/data/view/41601

--- [페이지 1, 상품 2] 상세 페이지: https://kdx.kr/data/view/41600

--- [페이지 1, 상품 3] 상세 페이지: https://kdx.kr/data/view/41599

--- [페이지 1, 상품 4] 상세 페이지: https://kdx.kr/data/view/41475

✅ [페이지 2] 수집 시작
상품 링크 개수: 4

--- [페이지 2, 상품 1] 상세 페이지: https://kdx.kr/data/view/41474

--- [페이지 2, 상품 2] 상세 페이지: https://kdx.kr/data/view/41473

--- [페이지 2, 상품 3] 상세 페이지: https://kdx.kr/data/view/41414

--- [페이지 2, 상품 4] 상세 페이지: https://kdx.kr/data/view/41413

✅ [페이지 3] 수집 시작
상품 링크 개수: 4

--- [페이지 3, 상품 1] 상세 페이지: https://kdx.kr/data/view/41412

--- [페이지 3, 상품 2] 상세 페이지: https://kdx.kr/data/view/41287

--- [페이지 3, 상품 3] 상세 페이지: https://kdx.kr/data/view/41286

--- [페이지 3, 상품 4] 상세 페이지: https://kdx.kr/data/view/41285

✅ [페이지 4] 수집 시작
상품 링크 개수: 4

--- [페이지 4, 상품 1] 상세 페이지: https://kdx.kr/data/view/41193

--- [페이지 4, 상품 2] 상세 페이지: https://kdx.kr/data/view/41192

--- [페이지 4, 상품 3] 상세 페이지: https://kdx.kr/data/view/41191

-

In [28]:
import asyncio
import json
import re
from playwright.async_api import async_playwright

# ✅ 여기에 직접 제공할 리스트 입력
start_urls = [
    "https://kdx.kr/data/product-list?specs_id=MA17200004&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200003&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200001&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190001&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190002&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190003&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190004&corp_id=CORP000036&category_id=CA000001%7CCA000002",
    "https://kdx.kr/data/product-list?specs_id=MA08210022&corp_id=CORP000007&category_id=CA000012",
    "https://kdx.kr/data/product-list?specs_id=MA04240002&corp_id=CORP000004&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA55230002&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA55230001&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA57240003&corp_id=CORP000050&category_id=CA000010",
    "https://kdx.kr/data/product-list?specs_id=MA20230001&corp_id=CORP000017&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA57240001&corp_id=CORP000050&category_id=CA000010",
    "https://kdx.kr/data/product-list?specs_id=MA55240002&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA19220002&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA54230001&corp_id=CORP000047&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA54230003&corp_id=CORP000047&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA19220003&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA19230002&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA20210011&corp_id=CORP000017&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA02200003&corp_id=CORP000002&category_id=CA000004"
]

base_url = "https://kdx.kr"

async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        output_file = "output.jsonl"
        with open(output_file, "w", encoding="utf-8") as f:

            for list_idx, start_url in enumerate(start_urls, start=1):
                print(f"\n✅ [리스트 {list_idx}] URL 접근: {start_url}")
                await page.goto(start_url)
                await page.wait_for_timeout(3000)

                a_tags = await page.query_selector_all("ul.card-list > a")
                print(f"상품 링크 개수: {len(a_tags)}")

                links = []
                for a in a_tags:
                    href = await a.get_attribute("href")
                    if href:
                        full_url = base_url + href
                        links.append(full_url)

                for detail_idx, detail_url in enumerate(links, start=1):
                    print(f"\n--- [리스트 {list_idx}, 상품 {detail_idx}] 상세 페이지: {detail_url}")
                    await page.goto(detail_url)
                    await page.wait_for_timeout(1500)

                    try:
                        purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                        purchase_text = await purchase_elem.inner_text()
                        price_num = int(re.sub(r"[^\d]", "", purchase_text))
                    except:
                        price_num = 0

                    try:
                        th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                        schema_columns = [await th.inner_text() for th in th_elements]
                        schema_columns = [col.strip() for col in schema_columns]
                    except:
                        schema_columns = []

                    try:
                        name_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(2) > td")
                        name_text = await name_elem.inner_text()
                    except:
                        name_text = "N/A"

                    try:
                        category_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(1) > td")
                        category_text = await category_elem.inner_text()
                    except:
                        category_text = "N/A"

                    try:
                        format_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(6) > td")
                        format_text = await format_elem.inner_text()
                    except:
                        format_text = "N/A"

                    try:
                        size_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(7) > td")
                        size_text = await size_elem.inner_text()
                    except:
                        size_text = "N/A"

                    data = {
                        "columns": schema_columns,
                        "price": price_num,
                        "name": name_text.strip(),
                        "category": category_text.strip(),
                        "format": format_text.strip(),
                        "size": size_text.strip(),
                        "url": detail_url
                    }

                    f.write(json.dumps(data, ensure_ascii=False) + "\n")

        await browser.close()
        print(f"\n✅ 전체 리스트 크롤링 및 파일 저장 완료 ➜ {output_file}")

await run()



✅ [리스트 1] URL 접근: https://kdx.kr/data/product-list?specs_id=MA17200004&corp_id=CORP000036&category_id=CA000002


ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


상품 링크 개수: 4

--- [리스트 1, 상품 1] 상세 페이지: https://kdx.kr/data/view/41607

--- [리스트 1, 상품 2] 상세 페이지: https://kdx.kr/data/view/41606

--- [리스트 1, 상품 3] 상세 페이지: https://kdx.kr/data/view/41605

--- [리스트 1, 상품 4] 상세 페이지: https://kdx.kr/data/view/41481

✅ [리스트 2] URL 접근: https://kdx.kr/data/product-list?specs_id=MA17200003&corp_id=CORP000036&category_id=CA000002
상품 링크 개수: 4

--- [리스트 2, 상품 1] 상세 페이지: https://kdx.kr/data/view/41604

--- [리스트 2, 상품 2] 상세 페이지: https://kdx.kr/data/view/41603

--- [리스트 2, 상품 3] 상세 페이지: https://kdx.kr/data/view/41602

--- [리스트 2, 상품 4] 상세 페이지: https://kdx.kr/data/view/41478

✅ [리스트 3] URL 접근: https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002
상품 링크 개수: 4

--- [리스트 3, 상품 1] 상세 페이지: https://kdx.kr/data/view/41601

--- [리스트 3, 상품 2] 상세 페이지: https://kdx.kr/data/view/41600

--- [리스트 3, 상품 3] 상세 페이지: https://kdx.kr/data/view/41599

--- [리스트 3, 상품 4] 상세 페이지: https://kdx.kr/data/view/41475

✅ [리스트 4] URL 접근: https://kdx.kr/data/produ

In [30]:
import asyncio
import json
import re
from playwright.async_api import async_playwright

# ✅ 여기에 직접 제공할 리스트 입력
start_urls = [
    "https://kdx.kr/data/product-list?specs_id=MA17200004&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200003&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200002&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17200001&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190001&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190002&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190003&corp_id=CORP000036&category_id=CA000002",
    "https://kdx.kr/data/product-list?specs_id=MA17190004&corp_id=CORP000036&category_id=CA000001%7CCA000002",
    "https://kdx.kr/data/product-list?specs_id=MA08210022&corp_id=CORP000007&category_id=CA000012",
    "https://kdx.kr/data/product-list?specs_id=MA04240002&corp_id=CORP000004&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA55230002&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA55230001&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA57240003&corp_id=CORP000050&category_id=CA000010",
    "https://kdx.kr/data/product-list?specs_id=MA20230001&corp_id=CORP000017&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA57240001&corp_id=CORP000050&category_id=CA000010",
    "https://kdx.kr/data/product-list?specs_id=MA55240002&corp_id=CORP000048&category_id=CA000003",
    "https://kdx.kr/data/product-list?specs_id=MA19220002&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA54230001&corp_id=CORP000047&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA54230003&corp_id=CORP000047&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA19220003&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA19230002&corp_id=CORP000016&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA20210011&corp_id=CORP000017&category_id=CA000001",
    "https://kdx.kr/data/product-list?specs_id=MA02200003&corp_id=CORP000002&category_id=CA000004"
]

base_url = "https://kdx.kr"


async def run():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        output_file = "output.jsonl"
        with open(output_file, "w", encoding="utf-8") as f:

            for list_idx, start_url in enumerate(start_urls, start=1):
                print(f"\n✅ [리스트 {list_idx}] URL 접근: {start_url}")
                await page.goto(start_url)
                await page.wait_for_timeout(3000)

                all_links = []

                # 페이지네이션 루프
                while True:
                    a_tags = await page.query_selector_all("ul.card-list > a")
                    print(f"상품 링크 개수 (현재 페이지): {len(a_tags)}")

                    for a in a_tags:
                        href = await a.get_attribute("href")
                        if href:
                            full_url = base_url + href
                            all_links.append(full_url)

                    # 다음 페이지 버튼 확인
                    next_button = await page.query_selector("#pagination > a.tui-page-btn.tui-next > span")
                    if next_button:
                        parent_button = await next_button.evaluate_handle("el => el.parentElement")
                        disabled = await parent_button.get_attribute("class")
                        if disabled and "disabled" in disabled:
                            break
                        await parent_button.click()
                        await page.wait_for_timeout(2000)
                    else:
                        break

                print(f"총 수집된 링크 개수: {len(all_links)}")

                for detail_idx, detail_url in enumerate(all_links, start=1):
                    print(f"\n--- [리스트 {list_idx}, 상품 {detail_idx}] 상세 페이지: {detail_url}")
                    await page.goto(detail_url)
                    await page.wait_for_timeout(1500)

                    try:
                        purchase_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-right > div.purchaseIs > p")
                        purchase_text = await purchase_elem.inner_text()
                        price_num = int(re.sub(r"[^\d]", "", purchase_text))
                    except:
                        price_num = 0

                    try:
                        th_elements = await page.query_selector_all("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > div.table-wrap.preview_wrap > div.sample_data > table > thead > tr > th")
                        schema_columns = [await th.inner_text() for th in th_elements]
                        schema_columns = [col.strip() for col in schema_columns]
                    except:
                        schema_columns = []

                    try:
                        name_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(2) > td")
                        name_text = await name_elem.inner_text()
                    except:
                        name_text = "N/A"

                    try:
                        category_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(1) > td")
                        category_text = await category_elem.inner_text()
                    except:
                        category_text = "N/A"

                    try:
                        format_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(6) > td")
                        format_text = await format_elem.inner_text()
                    except:
                        format_text = "N/A"

                    try:
                        size_elem = await page.query_selector("#dataView > div.container > div.detail-wrap > div.item-left > div.inner > table:nth-child(2) > tbody > tr:nth-child(7) > td")
                        size_text = await size_elem.inner_text()
                    except:
                        size_text = "N/A"

                    data = {
                        "columns": schema_columns,
                        "price": price_num,
                        "name": name_text.strip(),
                        "category": category_text.strip(),
                        "format": format_text.strip(),
                        "size": size_text.strip(),
                        "url": detail_url
                    }

                    f.write(json.dumps(data, ensure_ascii=False) + "\n")

        await browser.close()
        print(f"\n✅ 전체 리스트 크롤링 및 파일 저장 완료 ➜ {output_file}")

await run()


✅ [리스트 1] URL 접근: https://kdx.kr/data/product-list?specs_id=MA17200004&corp_id=CORP000036&category_id=CA000002
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 4
상품 링크 개수 (현재 페이지): 1
총 수집된 링크 개수: 157

--- [리스트 1, 상품 1] 상세 페이지: http